In [1]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import json

import time # importing time module

In [2]:
with open('APIkeys.json') as f:
    keys = json.load(f)
    omdbapi = keys['OMDBapi']

In [3]:
serviceurl = 'http://www.omdbapi.com/?'
apikey = '&apikey='+omdbapi

## Function for printing JSON dataset

In [4]:
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for k in list_keys:
        if k in list(json_data.keys()):
            print(f"{k}: {json_data[k]}")
    print("-"*50)

## Function to download poster

In [5]:
def save_poster(json_data):
    import os
    title = json_data['Title']
    poster_url = json_data['Poster']
    # Splits the poster url by '.' and picks up the last string as file extension
    poster_file_extension=poster_url.split('.')[-1]
    # Reads the image file from web
    poster_data = urllib.request.urlopen(poster_url).read()
        
    savelocation=os.getcwd()+'\\'+'Posters'+'\\'
    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    f=open(filename,'wb')
    f.write(poster_data)
    f.close()

## Function to create/update the local movie database with the data retreived from the web


Saves the movie data (Title, Year, Runtime, Country, Metascore, and IMDB rating) into a local SQLite database called 'movies.sqlite'

In [6]:
#this is the query string
#http://www.omdbapi.com/?t=Toy+Story&y=1995

In [7]:
def save_in_database(json_data):
    
    filename = 'output/IMDB.sqlite'
    
    import sqlite3
    conn = sqlite3.connect(str(filename))
    cur=conn.cursor()
    
    title = json_data['Title']
    # Goes through the json dataset and extracts information if it is available
    if json_data['Year']!='N/A':
        year = json_data['Year']
    else:
        year='NA'
    if json_data['Rated']!='N/A':
        rated = json_data['Rated']
    else:
        rated='NA'
    if json_data['Released']!='N/A':
        released = json_data['Released']
    else:
        released='NA'
    if json_data['Runtime']!='N/A':
        runtime = int(json_data['Runtime'].split()[0])
    else:
        runtime=-1
    if json_data['Awards']!='N/A':
        awards = json_data['Awards']
    else:
        awards='NA'
    if json_data['Metascore']!='N/A':
        metascore = float(json_data['Metascore'])
    else:
        metascore=-1
    if json_data['imdbRating']!='N/A':
        imdb_rating = float(json_data['imdbRating'])
    else:
        imdb_rating=-1
    if json_data['imdbVotes']!='N/A':
        imdb_votes = json_data['imdbVotes']
    else:
        imdb_votes=-1
    if json_data['imdbID']!='N/A':
        imdbId = json_data['imdbID']
    
    
    # SQL commands
    cur.execute('''CREATE TABLE IF NOT EXISTS MovieInfo 
    (Title TEXT, Year TEXT, Rated TEXT, Released TEXT, Runtime INTEGER, Awards TEXT, Metascore REAL, IMDBRating REAL, imdbVotes TEXT, imdbID TEXT)''')
    
    cur.execute('SELECT Title FROM MovieInfo WHERE Title = ? ', (title,))
    row = cur.fetchone()
    
    if row is None:
        cur.execute('''INSERT INTO MovieInfo (Title, Year, Rated, Released, Runtime, Awards, Metascore, IMDBRating, imdbVotes, imdbID )
                VALUES (?,?,?,?,?,?,?,?,?,?)''', (title,year,rated,released,runtime,awards,metascore,imdb_rating,imdb_votes,imdbId))
    else:
        print("Record already found. No update made.")
    
    # Commits the change and close the connection to the database
    conn.commit()
    conn.close()

## Function to print contents of the local database

In [8]:
def print_database(database):
    
    import sqlite3
    conn = sqlite3.connect(str(database))
    cur=conn.cursor()
    
    for row in cur.execute('SELECT * FROM MovieInfo'):
        print(row)
    conn.close()

## Function to save the database content in an CSV file

In [9]:
def save_in_csv(filename, database):
    
    if filename.split('.')[-1]!='csv':
        print ("Filename does not have correct extension. Please try again")
        return None
    
    import pandas as pd
    import sqlite3
    
    conn = sqlite3.connect(str(database))
    #cur=conn.cursor()
    
    df=pd.read_sql_query("SELECT * FROM MovieInfo", conn)
    conn.close()
    
    df.to_csv(filename, index = None, header=True)

## Function to search for information about a movie

In [26]:
def search_movie(m_title,year):
    if len(title) < 1 or title=='quit': 
        print("Goodbye now...")
        return None

    try:
        url = serviceurl + urllib.parse.urlencode({'t': title})+ '&' + urllib.parse.urlencode({'y': year}) + apikey
        
        print(f'Retrieving the data of "{title}" now... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            save_in_database(json_data)
            
        else:
            try:
                
                #try again this time with only 2 first characters from the movie title
            
                first = title.split(' ')[0].lower()
                first = first.ljust(len(first)+1)
                second = title.split(' ')[1].lower()
                m_title = first+second
                print(m_title)
                print(year)
                #title = first + ' '+ second
                print('--------2nd try: 2 words title --------')

                url = serviceurl + urllib.parse.urlencode({'t': m_title})+ '&' + urllib.parse.urlencode({'y': year}) + apikey
                time.sleep(5)
                save_in_database(json_data)
            except:
                print("Error: ",json_data['Error'])
            
    
    except urllib.error.URLError as e:
        print(f"ERROR: URL Error {e.reason}")

## Load Disney titles to get all the information through function search_movie

In [18]:
#Load Disney titles

disney_df = pd.read_csv('output/2021_disney_movies_total_gross.csv')
disney_df['year'] = pd.DatetimeIndex(disney_df['release_date']).year
disney_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


In [19]:
disney_df.count()

movie_title                 618
year                        618
release_date                618
total_gross                 618
inflation_adjusted_gross    618
dtype: int64

## === Test only STARTS here ===

In [37]:
#test with this df to see if it works as free API call is only 1000 calls per day
test_df = disney_df.loc[disney_df['movie_title'].isin(['101 Dalmatians','1492: Conquest of Paradise'])]
test_df

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759


In [42]:
# #TEST BLOCK: For smaller DF due to 1000 triggers a day only
# #query for all the movies in our disney list and get the latest information from IMDB

# for i in test_df.index: 
#     title = test_df['movie_title'][i].lower()
#     year = test_df['year'][i]
#     print (test_df['movie_title'][i])
#     print (test_df['year'][i])
#     print ('=====================================')
#     search_movie(title,year)
#     time.sleep(10)

101 Dalmatians
1961
Retrieving the data of "101 dalmatians" now... 
101 dalmatians
1961
--------2nd try: 2 words title --------
Error:  Movie not found!
101 Dalmatians
1996
Retrieving the data of "101 dalmatians" now... 
--------------------------------------------------
Title: 101 Dalmatians
Year: 1996
Rated: G
Released: 27 Nov 1996
Runtime: 103 min
Genre: Adventure, Comedy, Crime
Director: Stephen Herek
Writer: Dodie Smith, John Hughes
Actors: Glenn Close, Jeff Daniels, Joely Richardson
Plot: An evil high-fashion designer plots to steal Dalmatian puppies in order to make an extravagant fur coat, but instead creates an extravagant mess.
Language: English, Spanish
Country: United States, United Kingdom
Awards: Nominated for 1 BAFTA Film Award3 wins & 10 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '41%'}, {'Source': 'Metacritic', 'Value': '49/100'}]
Metascore: 49
imdbRating: 5.7
imdbVotes: 109,762
imdbID: t

## === Test only ENDS here ===

In [43]:
#query for all the movies in our disney list and get the latest information from IMDB

for i in disney_df.index: 
    title = disney_df['movie_title'][i].lower()
    year = disney_df['year'][i]
    print (disney_df['movie_title'][i])
    print (disney_df['year'][i])
    print ('=====================================')
    search_movie(title,year)
    time.sleep(10)

101 Dalmatians
1961
Retrieving the data of "101 dalmatians" now... 
101 dalmatians
1961
--------2nd try: 2 words title --------
Error:  Movie not found!
101 Dalmatians
1996
Retrieving the data of "101 dalmatians" now... 
--------------------------------------------------
Title: 101 Dalmatians
Year: 1996
Rated: G
Released: 27 Nov 1996
Runtime: 103 min
Genre: Adventure, Comedy, Crime
Director: Stephen Herek
Writer: Dodie Smith, John Hughes
Actors: Glenn Close, Jeff Daniels, Joely Richardson
Plot: An evil high-fashion designer plots to steal Dalmatian puppies in order to make an extravagant fur coat, but instead creates an extravagant mess.
Language: English, Spanish
Country: United States, United Kingdom
Awards: Nominated for 1 BAFTA Film Award3 wins & 10 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '41%'}, {'Source': 'Metacritic', 'Value': '49/100'}]
Metascore: 49
imdbRating: 5.7
imdbVotes: 109,762
imdbID: t

A Civil Action
1998
Retrieving the data of "a civil action" now... 
--------------------------------------------------
Title: A Civil Action
Year: 1998
Rated: PG-13
Released: 08 Jan 1999
Runtime: 115 min
Genre: Drama
Director: Steven Zaillian
Writer: Jonathan Harr, Steven Zaillian
Actors: John Travolta, Robert Duvall, Kathleen Quinlan
Plot: A tenacious lawyer takes on a case involving a major company responsible for causing several people to be diagnosed with leukemia due to the town's water supply being contaminated, at the risk of bankrupting his firm and career.
Language: English
Country: United States
Awards: Nominated for 2 Oscars. 5 wins & 10 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '62%'}, {'Source': 'Metacritic', 'Value': '68/100'}]
Metascore: 68
imdbRating: 6.6
imdbVotes: 28,628
imdbID: tt0120633
--------------------------------------------------
Record already found. No update made.
A Far Off 

A Thousand Acres
1997
Retrieving the data of "a thousand acres" now... 
--------------------------------------------------
Title: A Thousand Acres
Year: 1997
Rated: R
Released: 19 Sep 1997
Runtime: 105 min
Genre: Drama
Director: Jocelyn Moorhouse
Writer: Jane Smiley, Laura Jones
Actors: Michelle Pfeiffer, Jessica Lange, Jennifer Jason Leigh
Plot: A family is torn apart after a father gives his valuable farm land to his daughters.
Language: English
Country: United States
Awards: 2 wins & 1 nomination
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '24%'}]
Metascore: N/A
imdbRating: 6.1
imdbVotes: 5,485
imdbID: tt0120323
--------------------------------------------------
A Wrinkle in Time
2018
Retrieving the data of "a wrinkle in time" now... 
--------------------------------------------------
Title: A Wrinkle in Time
Year: 2018
Rated: PG
Released: 09 Mar 2018
Runtime: 109 min
Genre: Adventure, Family, Fantasy
Director: Ava DuVer

Aliens of the Deep
2005
Retrieving the data of "aliens of the deep" now... 
--------------------------------------------------
Title: Aliens of the Deep
Year: 2005
Rated: G
Released: 25 Aug 2005
Runtime: 100 min
Genre: Documentary, Family
Director: James Cameron, Steven Quale
Writer: N/A
Actors: James Cameron, Anatoly M. Sagalevitch, Genya Chernaiev
Plot: James Cameron teams up with NASA scientists to explore the Mid-Ocean Ridge, a submerged chain of mountains that band the Earth and are home to some of the planet's most unique life forms.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '84%'}, {'Source': 'Metacritic', 'Value': '71/100'}]
Metascore: 71
imdbRating: 6.4
imdbVotes: 2,996
imdbID: tt0417415
--------------------------------------------------
Alive
1993
Retrieving the data of "alive" now... 
--------------------------------------------------
Title: Alive
Year: 1993


Ant-Man
2015
Retrieving the data of "ant-man" now... 
--------------------------------------------------
Title: Ant-Man
Year: 2015
Rated: PG-13
Released: 17 Jul 2015
Runtime: 117 min
Genre: Action, Adventure, Comedy
Director: Peyton Reed
Writer: Edgar Wright, Joe Cornish, Adam McKay
Actors: Paul Rudd, Michael Douglas, Corey Stoll
Plot: Armed with a super-suit with the astonishing ability to shrink in scale but increase in strength, cat burglar Scott Lang must embrace his inner hero and help his mentor, Dr. Hank Pym, pull off a plan that will save the world.
Language: English
Country: United States
Awards: Nominated for 1 BAFTA Film Award4 wins & 34 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '83%'}, {'Source': 'Metacritic', 'Value': '64/100'}]
Metascore: 64
imdbRating: 7.3
imdbVotes: 635,594
imdbID: tt0478970
--------------------------------------------------
Record already found. No update made.
Ant-Man a

...

Tombstone
1993
Retrieving the data of "tombstone" now... 
--------------------------------------------------
Title: Tombstone
Year: 1993
Rated: R
Released: 25 Dec 1993
Runtime: 130 min
Genre: Action, Biography, Drama
Director: George P. Cosmatos, Kevin Jarre
Writer: Kevin Jarre
Actors: Kurt Russell, Val Kilmer, Sam Elliott
Plot: A successful lawman's plans to retire anonymously in Tombstone, Arizona are disrupted by the kind of outlaws he was famous for eliminating.
Language: English, Latin, Spanish
Country: United States
Awards: 2 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '74%'}, {'Source': 'Metacritic', 'Value': '50/100'}]
Metascore: 50
imdbRating: 7.8
imdbVotes: 141,590
imdbID: tt0108358
--------------------------------------------------
Tomorrowland
2015
Retrieving the data of "tomorrowland" now... 
--------------------------------------------------
Title: Tomorrowland
Year: 2015
Rated: PG
Released: 22 May

Tron
1982
Retrieving the data of "tron" now... 
--------------------------------------------------
Title: TRON
Year: 1982
Rated: PG
Released: 09 Jul 1982
Runtime: 96 min
Genre: Action, Adventure, Sci-Fi
Director: Steven Lisberger
Writer: Steven Lisberger, Bonnie MacBird, Charles S. Haas
Actors: Jeff Bridges, Bruce Boxleitner, David Warner
Plot: A computer hacker is abducted into the digital world and forced to participate in gladiatorial games where his only chance of escape is with the help of a heroic security program.
Language: English
Country: United States
Awards: Nominated for 2 Oscars. 2 wins & 8 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '71%'}, {'Source': 'Metacritic', 'Value': '58/100'}]
Metascore: 58
imdbRating: 6.8
imdbVotes: 119,316
imdbID: tt0084827
--------------------------------------------------
Record already found. No update made.
Tron: Legacy
2010
Retrieving the data of "tron: legacy"

Up Close & Personal
1996
Retrieving the data of "up close & personal" now... 
--------------------------------------------------
Title: Up Close & Personal
Year: 1996
Rated: PG-13
Released: 01 Mar 1996
Runtime: 124 min
Genre: Drama, Romance
Director: Jon Avnet
Writer: Joan Didion, John Gregory Dunne, Alanna Nash
Actors: Robert Redford, Michelle Pfeiffer, Stockard Channing
Plot: An ambitious young woman, determined to build a career in television journalism, gets good advice from her first boss, and they fall in love.
Language: Spanish, English
Country: United States
Awards: Nominated for 1 Oscar. 3 wins & 3 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.2/10'}, {'Source': 'Rotten Tomatoes', 'Value': '31%'}]
Metascore: N/A
imdbRating: 6.2
imdbVotes: 14,349
imdbID: tt0118055
--------------------------------------------------
Valiant
2005
Retrieving the data of "valiant" now... 
--------------------------------------------------
Title: Valiant
Year: 2005
Rat

When in Rome
2010
Retrieving the data of "when in rome" now... 
--------------------------------------------------
Title: When in Rome
Year: 2010
Rated: PG-13
Released: 29 Jan 2010
Runtime: 91 min
Genre: Comedy, Romance
Director: Mark Steven Johnson
Writer: David Diamond, David Weissman
Actors: Kristen Bell, Josh Duhamel, Anjelica Huston
Plot: Beth is a young, ambitious New Yorker who is completely unlucky in love. However, on a whirlwind trip to Rome, she impulsively steals some coins from a reputed fountain of love, and is then aggressively pursued by a band of suitors.
Language: English, Italian, Ukrainian
Country: United States
Awards: 3 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '17%'}, {'Source': 'Metacritic', 'Value': '25/100'}]
Metascore: 25
imdbRating: 5.6
imdbVotes: 61,844
imdbID: tt1185416
--------------------------------------------------
Where the Heart Is
1990
Retrieving the data of "where the hea

Wreck-It Ralph
2012
Retrieving the data of "wreck-it ralph" now... 
--------------------------------------------------
Title: Wreck-It Ralph
Year: 2012
Rated: PG
Released: 02 Nov 2012
Runtime: 101 min
Genre: Animation, Adventure, Comedy
Director: Rich Moore
Writer: Rich Moore, Phil Johnston, Jim Reardon
Actors: John C. Reilly, Jack McBrayer, Jane Lynch
Plot: A video game villain wants to be a hero and sets out to fulfill his dream, but his quest brings havoc to the whole arcade where he lives.
Language: English
Country: United States
Awards: Nominated for 1 Oscar. 33 wins & 42 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '87%'}, {'Source': 'Metacritic', 'Value': '72/100'}]
Metascore: 72
imdbRating: 7.7
imdbVotes: 406,874
imdbID: tt1772341
--------------------------------------------------
X Games 3D: The Movie
2009
Retrieving the data of "x games 3d: the movie" now... 
--------------------------------------

In [ ]:
disney_df.loc[disney_df['movie_title'] =='Endurance']

In [ ]:
# #Resolving issues with the parameters and starting from where we stopped and continue to the end of list

# for i in range(243,len(disney_df)): 
#     title = disney_df['movie_title'][i]
#     year = disney_df['year'][i]
#     print (disney_df['movie_title'][i])
#     print (disney_df['year'][i])
#     print ('=====================================')
#     search_movie(title,year)
#     time.sleep(10)

In [ ]:
# #Show downloaded poster of Titanic
# from IPython.display import Image
# Image("Posters/Toy Story.jpg")

## Print the content of the local database

In [44]:
print_database('output/IMDB.sqlite')

('20,000 Leagues Under the Sea', '1954', 'G', '20 Jul 1955', 127, 'Won 2 Oscars. 5 wins & 3 nominations total', 83.0, 7.2, '33,019', 'tt0046672')
('3 Ninjas', '1992', 'PG', '07 Aug 1992', 84, '1 nomination', 36.0, 5.3, '21,478', 'tt0103596')
('A Civil Action', '1998', 'PG-13', '08 Jan 1999', 115, 'Nominated for 2 Oscars. 5 wins & 10 nominations total', 68.0, 6.6, '28,628', 'tt0120633')
("A Kid in King Arthur's Court", '1995', 'PG', '11 Aug 1995', 89, '1 nomination', 34.0, 4.8, '6,143', 'tt0113538')
('A Low Down Dirty Shame', '1994', 'R', '23 Nov 1994', 108, '1 nomination', -1.0, 5.9, '7,204', 'tt0110399')
('Alive', '1993', 'R', '15 Jan 1993', 120, '1 win & 1 nomination', 56.0, 7.1, '54,692', 'tt0106246')
('An Innocent Man', '1989', 'R', '06 Oct 1989', 113, 'NA', 39.0, 6.5, '6,813', 'tt0097579')
('Angie', '1994', 'R', '04 Mar 1994', 107, '1 nomination', -1.0, 5.5, '2,193', 'tt0109129')
('Ant-Man', '2015', 'PG-13', '17 Jul 2015', 117, 'Nominated for 1 BAFTA Film Award4 wins & 34 nominati

## Save the database content into an Excel file

In [45]:
save_in_csv('output/IMDB_movies.csv','output/IMDB.sqlite')

In [46]:
#check the CSV is exported correctly
df=pd.read_csv('output/IMDB_movies.csv')
df

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,"33,019",tt0046672
1,3 Ninjas,1992,PG,07 Aug 1992,84,1 nomination,36.0,5.3,"21,478",tt0103596
2,A Civil Action,1998,PG-13,08 Jan 1999,115,Nominated for 2 Oscars. 5 wins & 10 nomination...,68.0,6.6,"28,628",tt0120633
3,A Kid in King Arthur's Court,1995,PG,11 Aug 1995,89,1 nomination,34.0,4.8,"6,143",tt0113538
4,A Low Down Dirty Shame,1994,R,23 Nov 1994,108,1 nomination,-1.0,5.9,"7,204",tt0110399
...,...,...,...,...,...,...,...,...,...,...
544,You Again,2010,PG,24 Sep 2010,105,2 wins,28.0,5.8,"51,344",tt1414382
545,The Young Black Stallion,2003,G,25 Dec 2003,50,1 nomination,51.0,5.4,"1,163",tt0318850
546,Zokkomon,2011,PG,22 Apr 2011,109,NaN,-1.0,4.1,400,tt1605790
547,Zootopia,2016,PG,04 Mar 2016,108,Won 1 Oscar. 47 wins & 72 nominations total,78.0,8.0,"477,250",tt2948356


In [47]:
df.count()

Title         549
Year          549
Rated         544
Released      548
Runtime       549
Awards        453
Metascore     549
IMDBRating    549
imdbVotes     549
imdbID        549
dtype: int64

In [49]:
df[df.Title.str.startswith('Star')]

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
116,Star Wars: Episode VIII - The Last Jedi,2017,PG-13,15 Dec 2017,152,Nominated for 4 Oscars. 25 wins & 99 nominatio...,84.0,6.9,"606,579",tt2527336
408,Star Wars: Episode VII - The Force Awakens,2015,PG-13,18 Dec 2015,138,Nominated for 5 Oscars. 62 wins & 138 nominati...,80.0,7.9,"904,569",tt2488496
